In [1]:
from datetime import datetime, timedelta
from processor.processor import Processor as p
from tqdm import tqdm
from alpaca_api.alpaca_api import AlpacaApi
from backtester.backtester_lite import BacktesterLite
from time import sleep
import pandas as pd
import json



In [2]:
alp = AlpacaApi()

In [3]:
end = (datetime.now()-timedelta(days=1)).strftime("%Y-%m-%d")
start = (datetime.now()-timedelta(days=1) - timedelta(days=10)).strftime("%Y-%m-%d")

In [4]:
sp500 = pd.read_csv("sp500.csv").rename(columns={"Symbol":"ticker"})

In [10]:
parameter = {
'strategy': 'window',
'value': True,
'lookback': 5,
'holding_period': 1,
'ceiling': 10,
'floor': -10
}
iteration = 6
positions = 20
lookback = parameter["lookback"]

In [6]:
simulation = []
for ticker in tqdm(sp500["ticker"]):
    try:
        example = alp.get_ticker_data(ticker,start,end)
        ticker_data = example.df.reset_index().rename(columns={"symbol":"ticker","timestamp":"date","close":"adjclose"})[["date","ticker","adjclose"]]
        ticker_data = p.column_date_processing(ticker_data)
        ticker_data.sort_values("date",inplace=True)
        ticker_data = ticker_data[["date","ticker","adjclose"]]
        ticker_data["prev_close"] = ticker_data["adjclose"].shift(1)
        ticker_data[f"window_{lookback}"] = ticker_data["prev_close"].shift(lookback)
        ticker_data[f"rolling_{lookback}"] = ticker_data["prev_close"].rolling(lookback).mean()
        simulation.append(ticker_data.dropna())
#         sleep(1)
    except Exception as e:
        print(ticker,str(e))
final = pd.concat(simulation)

  1%|██▎                                                                                                                                                                   | 7/490 [00:00<00:41, 11.55it/s]

ABMD 'NoneType' object has no attribute 'items'


 37%|███████████████████████████████████████████████████████████▉                                                                                                        | 179/490 [00:11<00:20, 15.23it/s]

RE 'NoneType' object has no attribute 'items'


 40%|█████████████████████████████████████████████████████████████████▉                                                                                                  | 197/490 [00:13<00:18, 16.18it/s]

...

 73%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                                           | 359/490 [00:49<00:26,  4.99it/s]

PKI 'NoneType' object has no attribute 'items'


 90%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                | 442/490 [01:13<00:07,  6.58it/s]

TWTR 'NoneType' object has no attribute 'items'


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 490/490 [01:29<00:00,  5.49it/s]


In [7]:
final = final[final["date"]==final["date"].max()]

In [11]:
try:
    trades = BacktesterLite.backtest(positions,final.copy(),iteration,parameter,True)
except Exception as e:
    print(str(e))

In [12]:
trades

,date,ticker,signal,position,iteration,strategy,value,lookback,holding_period,ceiling,floor
0,2023-08-08 04:00:00,MKTX,0.101645,0,6,window,True,5,1,10,-10
1,2023-08-08 04:00:00,QCOM,0.107508,1,6,window,True,5,1,10,-10
2,2023-08-08 04:00:00,MCHP,0.107652,2,6,window,True,5,1,10,-10
3,2023-08-08 04:00:00,MGM,0.110941,3,6,window,True,5,1,10,-10
4,2023-08-08 04:00:00,JCI,0.111555,4,6,window,True,5,1,10,-10
5,2023-08-08 04:00:00,ANSS,0.115422,5,6,window,True,5,1,10,-10
6,2023-08-08 04:00:00,ENPH,0.120930,6,6,window,True,5,1,10,-10
7,2023-08-08 04:00:00,EXPE,0.139920,7,6,window,True,5,1,10,-10
8,2023-08-08 04:00:00,MRNA,0.162648,8,6,window,True,5,1,10,-10
9,2023-08-08 04:00:00,PYPL,0.176964,9,6,window,True,5,1,10,-10


In [14]:
account = alp.live_get_account()
cash = float(account.cash)
try:
    if trades.index.size > 0: 
        # executing orders
        for row in trades.iterrows():
            try:
                ticker = "BTC/USD" if row[1]["ticker"] == "BTC" else row[1]["ticker"]
                amount = round(cash / positions,2)
                print(ticker,amount)
#                 order_data.append(alp.live_market_order(ticker,amount))
#                 sleep(1)
            except Exception as e:
                print(str(e))
except Exception as e:
        print(str(e))

MKTX 24.97
QCOM 24.97
MCHP 24.97
MGM 24.97
JCI 24.97
ANSS 24.97
ENPH 24.97
EXPE 24.97
MRNA 24.97
PYPL 24.97
IPGP 24.97
RMD 24.97
NCLH 24.97
ETSY 24.97
ZBRA 24.97
PAYC 24.97
SEDG 24.97
FTNT 24.97
GNRC 24.97
DXC 24.97
